# Les requêtes imbriquées avec Neo4J
Ce rapport a pour objectif de montrer l'utilisation des requêtes imbriquées dans le système de gestion de base de donnée Neo4j.
Pour pouvoir executer le code présent dans les cellules suivantes il faut avoir neo4j lancé, utiliser un docker est possible avec : 

``` docker run --publish=7474:7474 --publish=7687:7687 -v $PWD/import:/var/lib/neo4j/import neo4j:4.1.3```


Il faut que le fichier small-pubg.csv soit present dans un dossier import a la racine du projet

## Un peu de contexte

Neo4j est un système de gestion de base de donnée orientée graphe. Ce type de système a la particularité d'utiliser des graphes pour représenter les données et ainsi permettre les formules connus de la théorie des graphes. De plus Neo4j est connu pour sa performance dans la rapidité des requêtes qui sont dans certains cas plus rapide que des simples requêtes SQL.Le langage de requête utilisé par Neo4j est le langage cypher reconnu pour permettre des requêtes clair et simple. 

Un autre avantage de ce type de base de donnée est la simplicité et la clareté des graphes produits qui peuvent être utilisé simplement par différents domaine métiers par exemple des analyste sans compêtences informatique nécessaire il suffit de regarder les relations entre les noeuds pour pouvoir comprendre facilement le modèle de donnée les relations présente et leurs implication. De plus neo4j permet grâce à son interface facilement maniable de mettre en évidence ou de déplacer certains noeuds, ainsi en clickant que un noeud l'on peut voir ses relations avec les autres et les déplacer pour clarifier la lecture. 

Les requêtes imbriquées (ou sous requête) permettent de faire une requête sur une requête et donc cibler plus simplement un resultat.
En SQL par exemple une requête de ce type ressemble à ça : ```SELECT a FROM b WHERE a IN (SELECT c FROM b) ```
A l'aide de la bibliothèque python py2neo nous allons vous montrer comment on peut utiliser ces requêtes. Nous utilisons la verion 4.1.3 de neo4j, en effet les sous requêtes sont assez récente dans le langage et il faut au moins utiliser la version 4.0.0.

## Installation

Nous utiliserons donc les librairies suivantes permettant de faire des requêtes à notre base neo4j depuis jupyter


```python pip3 install py2neo ```


```python pip3 install neo4jupyter ```


## Import des données pubg dans un graphe

### Connexion à la base neo4j

In [5]:
from py2neo import *
from neo4jupyter import *

graph = Graph('http://localhost', auth=('neo4j', 'neo4j*'))
print("connect")

connect


### Import des données pubg

Il faut que $neo4jhome/import/small-pubg sois défini

Ici Merge permet de créer des entités à partir du csv chargé nous créons donc trois entitées pour les joueurs les équipes et les matchs (un joueur appartient à une équipe)

In [6]:
loadQuery = '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///small-pubg.csv' AS pubg
WITH pubg
WHERE pubg.match_id IS NOT NULL AND pubg.player_name IS NOT NULL
MERGE (player:Player{
        assists:pubg.player_assists,
        name:pubg.player_name,
        kills:pubg.player_kills,
        surviveTime:pubg.player_survive_time,
        dmg:pubg.player_dmg,
        dbno:pubg.player_dbno,
        distRide:pubg.player_dist_ride,
        distWalk:pubg.player_dist_walk
        
        })
MERGE (match:Match{
         matchId:pubg.match_id,
         matchMode:pubg.match_mode
         })
MERGE (team:Team{
        teamId:pubg.team_id,
        teamPlacement:pubg.team_placement
        })
MERGE (player) -[:belongs_to]-> (team)
'''
graph.run(loadQuery)
print("ok")

ok


### Visualisation du graphe 

Cette cellule permet d'avoir un aperçu visuel du graphe produit, pour créer cette aperçu nous utilisons le script vis.py que nous avons importé dans notre dépot git depuis : https://github.com/nicolewhite/neo4j-jupyter/blob/master/scripts/vis.py

In [7]:
neo4jupyter.init_notebook_mode()
options = {"Player": "name", "Match": "matchId", "Team": "teamId"}
draw(graph, options)

<IPython.core.display.Javascript object>

## Présentation des requêtes imbriquées

Il existe deux manières de faire les requêtes imbriquées en neo4j :
- Les requêtes existentiel utilisant la clause cypher EXISTS{} dans la clause WHERE
- Les requêtes utilisant la clause CALL et permettant de retourner un résultat



### Requête existentiel

### Requête avec résultat

date
game_size
match_id
match_mode
party_size
player_assists
player_dbno
player_dist_ride
player_dist_walk
player_dmg
player_kills
player_name
player_survive_time
team_id
team_placement
